In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpmath import whitw
from scipy.integrate import solve_ivp
from scipy.integrate import odeint
from scipy.integrate import quad
import pandas as pd
import math
from scipy.interpolate import CubicSpline
from scipy.interpolate import RectBivariateSpline
from scipy.optimize import fsolve
from timer import Timer
import os
from importlib import reload
import utilities
reload(utilities)
from utilities import EndOfInflation

In [2]:
def ModeEoM(A, k, dphidt, dIdphi, a, sigmaE=0, sigmaB=0):
    #omega=1.
    dAdt = np.zeros(A.size)
    
    drag = a**(-alpha) * sigmaE
    dis1 = k * a**(alpha-1)
    dis2 = dIdphi * dphidt + a**(alpha)*sigmaB

    #positive helicity
    lam = 1.
    #Real Part
    dAdt[0] = A[1]*(k/a)*a**(alpha)
    dAdt[1] = -( drag * A[1] + (dis1  - lam * dis2) * A[0] )
    
    #Imaginary Part
    dAdt[2] = A[3]*(k/a)*a**(alpha)
    dAdt[3] = -( drag * A[3] + (dis1  - lam * dis2) * A[2] )
    
    
    #negative helicity
    lam = -1.
    #Real Part
    dAdt[4] = A[5]*(k/a)*a**(alpha)
    dAdt[5] = -( drag * A[5] + (dis1  - lam * dis2) * A[4] )
    
    #Imaginary Part
    dAdt[6] = A[7]*(k/a)*a**(alpha)
    dAdt[7] = -( drag * A[7] + (dis1  - lam * dis2) * A[6] )
    
    return dAdt

In [62]:
def ModeByMode(N0, Nmid, Nend, kh, dphidt, dIdphi, N, steps=800, cut=0.1,
               sigmaE=lambda t:0, sigmaB=lambda t:0, delta=lambda t:1., save=False, KDepDamp=False):
    #Convert N-input to t
    t = np.linspace(0,120,100)
    t0 = fsolve(lambda t: N(t) - N0, 1)[0]
    tmid = fsolve(lambda t: N(t) - Nmid, 45)[0]
    tend = fsolve(lambda t: N(t) - Nend, 100)[0]

    dt1 = 4*(tmid-t0)/(steps)
    dt2 = dt1/4
    ts1 = np.arange(t0, tmid-dt1, dt1)
    ts2 = np.arange(tmid, tend, dt2)
    ts = np.concatenate((ts1, ts2), axis=0)

    Ap = []
    dApdt = []
    Am = []
    dAmdt = []
    ks = []
    t2 = Timer()
    t2.start()

    a = lambda t: np.exp(N(t))
    
    for i in range(1,steps):
        s = ts[i]
        if (kh(s)/kh(ts[i-1]) > cut): 
            k = 10**(5/2)*kh(s)

            #t1 = Timer()
            ks.append(k)
        
            #t1.start()
            Aini = np.array([1., 0, -1/2*sigmaE(s)/k, -1., 1, 0, -1/2*sigmaE(s)/k, -1.])*np.sqrt(delta(s))
            ode = lambda x, y: ModeEoM(y, k, dphidt(x), dIdphi, a(x), sigmaE=sigmaE(x), sigmaB=sigmaB(x))
            
            sol = solve_ivp(ode, [s, ts[-1]], Aini, t_eval=ts[i:], method="RK45", atol=1e-8, rtol=1e-8)
            
            #vacuum will not contribute to integrals, so we set this part to 0 to have equal length arrays
            vac = list((1+0j)*np.ones((i)))
            dvacdt = list((0.-1j)*np.ones((i)))
            
            Aptmp = list(np.array(sol.y[0,:]) + np.array(sol.y[2,:])*1j)
            Ap.append([*vac, *Aptmp])
            
            dAptmp = list(np.array(sol.y[1,:]) + np.array(sol.y[3,:])*1j)
            dApdt.append([*dvacdt, *dAptmp])
            
            
            Amtmp = list(np.array(sol.y[4,:]) + np.array(sol.y[6,:])*1j)
            Am.append([*vac, *Amtmp])
            
            dAmtmp = list(np.array(sol.y[5,:]) + np.array(sol.y[7,:])*1j)
            dAmdt.append([*dvacdt, *dAmtmp])

            #t1.stop()
        else:
            print("skip")
            continue
            
    Ap = np.array(Ap)
    dApdt = np.array(dApdt)            
    Am = np.array(Am)
    dAmdt = np.array(dAmdt)
    
    t2.stop()
    
    if save:
        SaveMode(ts, ks, Ap, dApdt, Am, dAmdt, af)
                   
    
    return ts, ks, Ap, dApdt, Am, dAmdt

In [63]:
name = "_Beta25.0_SENone"

In [64]:
file = "Out/GEF" + name + ".dat"
beta = 25
Mpl = 1
alpha = 0
SE=False
input_df = pd.read_table(file, sep=",")
#data = input_df.values
data = dict(zip(input_df.columns[1:],input_df.values[1:,1:].T))
NR = data["N"]
tR = data["t"]
aR = data["a"]
khR = data["kh"]
dphidtR = data["dphi"]
Nf = CubicSpline(tR, NR)
dphidtf = CubicSpline(tR, dphidtR)
khf = CubicSpline(tR, khR)
#print( tR )
#print( NR )
#print( np.log( af(tR) ) )

ts, ks, Ap, dAp, Am, dAm = ModeByMode(1, 30, max(NR), khf, dphidtf, beta/Mpl, Nf, cut=0.01)

[1.53804155e-62 1.69184570e-61 1.70722612e-60 ... 1.19917835e+02
 1.19994717e+02 1.20000000e+02]
[1.53804155e-62 1.69184570e-61 1.70722612e-60 ... 6.23342847e+01
 6.23431531e+01 6.23437623e+01]
[ 0.          0.          0.         ... 62.33428467 62.34315305
 62.3437623 ]
Elapsed time: 829.2249 seconds
